In [92]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
import os
import random
import numpy as np
import cv2
from glob import glob
import re

#global declarations
img_size = 28
input_shape = (img_size, img_size, 1)

In [93]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=input_shape))
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model
create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               589952    
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
__________

In [94]:
def prepare_dataset(training_dir):
    viable_num=[str(n) for n in list(range(10))]
    files = glob(training_dir+'*.png')
    files = [f for f in files if f[8] in viable_num]
    random.shuffle(files)
    
    samples = []
    for f in files:
        label = f[8]
        
        src = cv2.imread(f, cv2.IMREAD_COLOR)
        if len(src.shape) != 2:
            gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
        else:
            gray = src
            
        _ , threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
        threshold = cv2.bitwise_not(threshold)
        threshold = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, np.ones((3,3)))
        
        samples.append((threshold, label))
       
    training_data = [i[0] for i in samples]
    training_data = np.asarray(training_data).astype('float32')
    training_data = np.expand_dims(training_data, axis=3)
    training_data /= 255
    training_labels = [i[1] for i in samples]
    training_labels = np.asarray(training_labels)
    training_labels = np_utils.to_categorical(training_labels, 10)
    return (training_data, training_labels)

In [96]:
def train_classifier():
    model = create_model()
    print('Model created')
    training_data, training_labels = prepare_dataset('symbols/')
    print('Dataset created')
    model.fit(training_data, training_labels, batch_size=64, epochs=5, verbose=1)
    print('Model trained')
    model.save('kares_model.mod')
    print('Model saved')
    
train_classifier()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               589952    
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
__________